In [1]:
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import csv

In [28]:
from models.HDAAGT import *
from train_test import *
from utilz.utils import *
import datetime
cwd = os.getcwd()

device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
with  open("/home/abdikhab/HDAAGT/configs/config.yaml") as file:
    config = yaml.safe_load(file)
cwd = os.getcwd()
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
ct = datetime.datetime.now().strftime("%m-%d-%H-%M") # Current time, used for saving the log
print(f"Total  # of GPUs {torch.cuda.device_count()}")
print(f"Using {device} device")


ZoneConf = Zoneconf(config['Zoneconf_path']) # This is the setting for zone configuration, you can find the image file at 'data/Zone Configuration.jpg'
# config non-indipendent parameters
config['NFeatures']= len(config["Columns_to_keep"]) # This goes to the data preparation process
config["input_size"] = config["NFeatures"] + 9 + 8 # This goes to the model, we add couple of more features to the input
config["output_size"] = len(config['xy_indx']) # The number of columns to predict
config['device'] = device
config['ct'] = ct
log_code = f"Learning rate: {config['learning_rate']}, Hidden size: {config['hidden_size']}, Batch size: {config['batch_size']}"
if config['verbal']:
    for arg in config.items(): # Print the arguments to the log file
        savelog(f"{arg[0]} = {arg[1]}", ct)
config['sos'] = torch.cat((torch.tensor([10,1016,1016,7,7,7,7]) , torch.zeros(17)), dim=0).repeat(config['Nnodes'], 1).to(device)
config['eos'] = torch.cat((torch.tensor([11,1020,1020,8,8,8,8]), torch.zeros(17)), dim=0).repeat(config['Nnodes'],1).to(device)

model = HDAAGT(config).to(device) # Here we define our model
savelog("Creating model from scratch",ct)
criterion = nn.CrossEntropyLoss()
model.load_state_dict(torch.load(config['Load_Model_Path'], map_location=device))

Total  # of GPUs 4
Using cuda:3 device
Creating model from scratch


/tmp/ipykernel_137394/1820070400.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(config['Load_Model_Path'], map_location=device))


<All keys matched successfully>

In [29]:
learning_rate, hidden_size, num_layers, batch_size, ct= 1e-4, 128, 1, 128, '02-04-13-46'

code = f"LR{int(learning_rate*100000)}_HS{hidden_size}_NL{num_layers}_BS{batch_size} {ct}"
# Predicteddata  = torch.load(os.path.join(os.getcwd(),'Pickled', code + 'Predicteddata.pt'), weights_only=True)
ActualTarget = torch.load(os.path.join(os.getcwd(),'Pickled', 'Test', 'Target.pt'), weights_only=True)
ActualScene = torch.load(os.path.join(os.getcwd(),'Pickled', 'Test', 'Scene.pt'), weights_only=True)
Adj_Mat_Scene = torch.load(os.path.join(os.getcwd(),'Pickled', 'Test', 'Adj_Mat_Scene.pt'), weights_only=True)
# Adj_Mat_Target = torch.load(os.path.join(os.getcwd(),'Pickled', 'Test', 'Adj_Mat_Target.pt'), weights_only=True)
# test_loss = torch.load(os.path.join(os.getcwd(),'Pickled', code + 'test_losses.pt'), weights_only=True)
epoch_losses = torch.load(os.path.join(cwd,'Pickled', f'epoch_losses.pt'), weights_only=True)

In [30]:
# from torchsummary import summary
from torchinfo import summary  # more flexible than torchsummary

device = torch.device('cuda:3')
def custom_summary(model, x, src_mask, adj_mat):
    model.eval()
    with torch.no_grad():
        output = model(x, src_mask, adj_mat)
    print(f"Output shape: {output.shape}")

# Dummy input example (adjust shapes accordingly)
x = attach_sos_eos(ActualScene[:1], config['sos'], config['eos'])
adj_mat = torch.cat((torch.ones_like(Adj_Mat_Scene[:1,:1]), Adj_Mat_Scene[:1], torch.ones_like(Adj_Mat_Scene[:1,:1])), dim=1)
src_mask = create_src_mask(x)
# custom_summary(model, x, src_mask, adj_mat)
summary(model, input_size=[x.size(), src_mask.size(), adj_mat.size()], device=device)

Layer (type:depth-idx)                        Output Shape              Param #
HDAAGT                                        [1, 17, 32, 2, 1024]      --
├─Encoder_DAAG: 1-1                           [32, 17, 384]             --
│    └─Positional_Encoding_Layer: 2-1         [32, 17, 384]             296,064
│    │    └─ModuleList: 3-1                   --                        262,144
│    │    └─DAAG_Layer: 3-2                   [1, 17, 32, 256]          211,456
│    │    └─TemporalConv: 3-3                 [32, 17, 128]             20,892
│    │    └─MultiheadAttention: 3-4           [32, 17, 384]             591,360
│    │    └─Dropout: 3-5                      [32, 17, 384]             --
│    └─Traffic_Encoding_Layer: 2-2            [32, 17, 384]             1,152
│    │    └─ModuleList: 3-6                   --                        1,248
│    │    └─LayerNorm: 3-7                    [1, 17, 32, 5]            10
│    │    └─Linear: 3-8                       [1, 17, 32, 128]   

In [31]:
from torchviz import make_dot
model.eval()
output = model(x, src_mask, adj_mat)

# Create graph
dot = make_dot(output, params=dict(model.named_parameters()))
dot.format = 'png'
dot.render('models/HDAAGT_graph', cleanup=True)

'models/HDAAGT_graph.png'

In [58]:
import matplotlib.pyplot as plt
import torch.nn.functional as F

batch = 32
Predicteddata = []
for n in range(0, ActualScene.size(0), batch):
    x, src_mask,adj_mat, target = prep_model_input(ActualScene[n:n+batch], Adj_Mat_Scene[n:n+batch], ActualTarget[n:n+batch], config['sos'], config['eos'], config)
    Predicteddata += model(x, src_mask, adj_mat)

Predicteddata = torch.stack(Predicteddata)
a = Predicteddata[:,:].softmax(dim=-1)
top_values, top_indices = torch.topk(a, k = 5, dim=-1)
pred_tar0 = top_indices[:,:,:,:,0]
pred_tar = (top_indices*top_values).sum(-1)/top_values.sum(-1)
print(pred_tar0.shape, pred_tar.shape)
true_loss = F.mse_loss(top_indices[:,1:-1,:,:,0], ActualTarget[:,:,:,1:3])
lvl2_loss = F.mse_loss(pred_tar0[:,1:-1], ActualTarget[:,:,:,1:3])
print(true_loss, lvl2_loss)
error = torch.sqrt(torch.pow((pred_tar0[:,1:-1]- ActualTarget[:,:,:,1:3]),2).sum(-1)).mean()
print(error)

torch.Size([390, 17, 32, 2]) torch.Size([390, 17, 32, 2])
tensor(2855.8521, device='cuda:3') tensor(2855.8521, device='cuda:3')
tensor(14.9209, device='cuda:3')


In [64]:

img0 = cv2.imread("data/no-car.JPG")
path = "Processed/results"
icon = cv2.imread("data/icon.png", cv2.IMREAD_UNCHANGED)  # Includes alpha
icon = cv2.resize(icon, (50, 50))

# Rotation parameters
(h, w) = icon.shape[:2]
center = (w // 2, h // 2)

for m in range(398):
    img = img0.copy()
    scene = ActualScene[m]
    tar = ActualTarget[m,-1]
    pred = pred_tar0[m,-2]
    for i in range(1, 15):
        angle = 180*torch.atan((scene[i,0,2]-scene[i-1,0,2])/(scene[i,0,1] - scene[i-1,0,1]))/torch.pi  # degrees
        # Rotate icon (preserving alpha channel)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated_icon = cv2.warpAffine(icon, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0,0,0,0))
        # Split rotated icon into BGR and alpha
        icon_rgb = rotated_icon[:, :, :3]
        alpha = rotated_icon[:, :, 3] / 255.0
        # Set position to place icon
        x_offset, y_offset = 10, 10
        # Define ROI on the background
        roi = background[y_offset:y_offset+h, x_offset:x_offset+w]

        # Blend icon into background
        for c in range(3):
            roi[:, :, c] = roi[:, :, c] * (1 - alpha) + icon_rgb[:, :, c] * alpha

        # Replace the region in the original image
        background[y_offset:y_offset+h, x_offset:x_offset+w] = roi

        egox, egoy = scene[:, i, 0], scene[:,i, 1]
        targetx, targety = tar[i, 0], tar[i, 1]
        # if dir != 0:
        for n in range(int(egox.size(0)-1)):
            if egox[n] > 10 and egox[n+1] > 10:
                cv2.line(img, (int(egox[n]), int(egoy[n])),(int(egox[n+1]), int(egoy[n+1])), color = rand_color[i], thickness= 4)
                # cv2.circle(img, (int(egox[n]), int(egoy[n])), 5, rand_color[i], -1)
        cv2.circle(img, (int(tar[i, 0]), int(tar[i, 1])), 10, rand_color[i], -1)
        cv2.rectangle(img, (int(pred[i, 0]), int(pred[i, 1])), (int(pred[i, 0]+15), int(pred[i, 1]+15)), rand_color[i], -1)
    
    cv2.imwrite(os.path.join(path, f"pred{m}.jpg"), img)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
print("Images created successfully")


# Load background and icon
background = cv2.imread("background.jpg")


# Resize if needed
icon = cv2.resize(icon, (50, 50))

# Rotation parameters
angle = 45  # degrees
(h, w) = icon.shape[:2]
center = (w // 2, h // 2)


# Show result
cv2.imshow("Rotated Icon on Background", background)
cv2.waitKey(0)
cv2.destroyAllWindows()


TypeError: Argument 'angle' can not be treated as a double

In [ ]:
rand_color = [
    (0, 0, 0),        # Black
    (0, 0, 255),      # Red
    (0, 255, 0),      # Green
    (255, 0, 0),      # Blue
    (0, 255, 255),    # Yellow
    (255, 255, 0),    # Cyan
    (255, 0, 255),    # Magenta
    (128, 0, 128),    # Purple
    (0, 165, 255),    # Orange
    (130, 0, 75),     # Indigo
    (128, 128, 0),    # Teal
    (0, 0, 128),      # Maroon
    (128, 0, 0),       # Navy
    (128, 128, 128),  # Gray
    (192, 192, 192),  # Silver
    (255, 255, 255),   # White
    (255, 165, 0),     # Brown
    (0, 128, 0),       # Olive
    (0, 128, 128),     # Aqua
    (128, 0, 128),     # Fuchsia
    (128, 128, 0),     # Lime
# Some more colors
    (50, 50, 50),      # Dark Gray
    (60, 120, 120),    # Sea Green
    (120, 30, 60),     # Crimson
    (100, 20, 120),    # Violet
    (10, 32, 64),       # Dark Blue
    (64,10, 32),        # Dark Red
    (10, 64, 32),       # Dark Green
    (16, 32, 64),       # Dark Aqua
    (32, 64, 16),       # Dark Lime
    (64, 32, 16),       # Dark Orange
    (64, 16, 32),       # Dark Pink

]

In [ ]:
import yaml
import re
from shapely.geometry import Point, Polygon
def Zoneconf1():
    ZoneConf = []
    with open('utilz/ZoneConf.yaml') as file:
        ZonesYML = yaml.load(file, Loader=yaml.FullLoader)
        #convert the string values to float
        for _, v in ZonesYML.items():
            lst = []
            for _, p  in v.items():    
                for x in p[0]:
                    b = re.split(r'[,()]',p[0][x])
                    lst.append((float(b[1]), float(b[2])))
            ZoneConf.append(lst)
    return ZoneConf

def zonefinder(BB, Zones):
    for i, zone in enumerate(Zones):
        Poly = Polygon(zone)
        if Poly.contains(Point(BB[0], BB[1])):
            return i
    return 100

In [ ]:
zoneconf = Zoneconf1()
zones = torch.empty(0,2)
zonesw0 = torch.empty(0,2)
err = 0
for m in range(440):
    tar = target[m,-1].to('cpu')
    pred = pred_tar[m,-4:-1].mean(0).to('cpu')
    for i in range(32):
        zt = zonefinder(tar[i],zoneconf)
        zp = zonefinder(pred[i], zoneconf)
        if zp == 100 and zt != 100:
            # print(pred[i], tar[i], zp, zt)
            err +=1
        else:
            zonesw0 = torch.cat((zonesw0,torch.tensor([[zp,zt]])), dim=0)
            if zt !=100:
                zones = torch.cat((zones,torch.tensor([[zp,zt]])), dim=0)

print(err)


In [ ]:
a = (zones[:,0]==zones[:,1]).sum()
b = zones.size(0)
print("Accuracy without zone 100 is ",a, b, a/b )
a = (zonesw0[:,0]==zonesw0[:,1]).sum()
b = zonesw0.size(0)
print("Accuracy with zone 100 is ",a, b, a/b )

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


# Compute the confusion matrix
cm = confusion_matrix(zones[:,1], zones[:,0])

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm )
disp = disp.plot(cmap=plt.cm.YlGn,values_format='g')
# disp.plot()
plt.xlabel('Predicted Zones')
plt.ylabel('True Zones')
plt.title('Confusion Matrix')
SMALL_SIZE = 8
MEDIUM_SIZE = 12
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)

In [ ]:
disp = disp.plot(cmap=plt.cm.YlGnBu,values_format='g')

In [ ]:

img0 = cv2.imread("/home/abdikhab/Traj_Pred/RawData/no-car.JPG")
path = "/home/abdikhab/New_Idea_Traj_Pred/results"
for m in range(440):
    img = img0.copy()
    scene = scene[m]
    tar = target[m,-1]
    pred = pred_tar0[m,-2]
    for i in range(32):
        
        egox, egoy = scene[:, i, 0], scene[:,i, 1]
        targetx, targety = tar[i, 0], tar[i, 1]
        # if dir != 0:
        for n in range(int(egox.size(0)-1)):
            if egox[n] > 10 and egox[n+1] > 10:
                cv2.line(img, (int(egox[n]), int(egoy[n])),(int(egox[n+1]), int(egoy[n+1])), color = rand_color[i], thickness= 4)
                # cv2.circle(img, (int(egox[n]), int(egoy[n])), 5, rand_color[i], -1)
        cv2.circle(img, (int(tar[i, 0]), int(tar[i, 1])), 10, rand_color[i], -1)
        cv2.rectangle(img, (int(pred[i, 0]), int(pred[i, 1])), (int(pred[i, 0]+15), int(pred[i, 1]+15)), rand_color[i], -1)
    
    cv2.imwrite(os.path.join(path, f"pred{m}.jpg"), img)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
print("Images created successfully")

In [ ]:
# Plotting the loss
trainloss = []
testloss = []
epochs = []
for obj in test_loss:
    trainloss.append(epoch_losses[obj[1]])
    testloss.append(obj[0])
    epochs.append(obj[1])
plt.plot(epochs, trainloss)
plt.title('Epoch Losses during batch Training')
plt.plot(epochs, testloss)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train Loss', 'Test Loss'])
plt.ylim(0, 100000)
plt.show()

***Now, we want to predict the future sequence using the best model***

In [ ]:
from utilz.utils import *
import torch.nn.functional as F

Best_Model=torch.load(os.path.join(os.getcwd(),'Processed', code + 'Bestmodel.pth'))
# Best_Model_dict = Best_Model.state_dict()

# Model = GGAT(args)
# Model.load_state_dict(Best_Model_dict)

In [ ]:
def eval(model, Scene, Adj_mat, h0, Nseq):
    # savelog("Starting testing phase", ct)
    model.eval()
    with torch.no_grad():
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)
        start_event.record()
        Out, Gruh0 = model(Scene, Adj_mat, h0, Nseq)
        end_event.record()
        torch.cuda.synchronize()
        inference_time = start_event.elapsed_time(end_event)
    return Out, Gruh0, inference_time

In [ ]:
def attach(NScene, Scenelet, Zonecft):
    x,y = NScene[:,:,0], NScene[:,:,1]

    dx = torch.diff(x, n=1, append=x[-1:,:], dim = 0).unsqueeze(2)
    dy = torch.diff(y, n=1, append=y[-1:,:], dim = 0).unsqueeze(2)
    heading = torch.atan2(dy, dx)
    xc = x - 512
    yc = y - 512
    Rc = torch.sqrt(xc**2 + yc**2).unsqueeze(2)
    SinX = torch.sin(xc/512).unsqueeze(2)
    CosY = torch.cos(yc/512).unsqueeze(2)
    SinY = torch.sin(yc/512).unsqueeze(2)
    CosX = torch.cos(xc/512).unsqueeze(2)
    Sin2X = torch.sin(2*xc/512).unsqueeze(2)
    Cos2X = torch.cos(2*xc/512).unsqueeze(2)
    Sin2Y = torch.sin(2*yc/512).unsqueeze(2)
    Cos2Y = torch.cos(2*yc/512).unsqueeze(2)


    NScene = torch.cat((NScene, Scenelet, dx, dy, heading, xc.unsqueeze(2),yc.unsqueeze(2), Rc,
                            SinX, CosX, SinY, CosY,
                            Sin2X, Cos2X, Sin2Y, Cos2Y), dim=2)
    for i in range(NScene.size(1)):
        NScene[:,i,9] = torch.tensor(zonefinder(NScene[0,i,:2].cpu().numpy(), Zonecft), device=NScene.device)
    return NScene

def Zoneconf():
    ZoneConf = []
    with open('utilz/ZoneConf.yaml') as file:
        ZonesYML = yaml.load(file, Loader=yaml.FullLoader)
        #convert the string values to float
        for _, v in ZonesYML.items():
            lst = []
            for _, p  in v.items():    
                for x in p[0]:
                    b = re.split(r'[,()]',p[0][x])
                    lst.append((float(b[1]), float(b[2])))
            ZoneConf.append(lst)
    return ZoneConf

def zonefinder(BB, Zones):
    for i, zone in enumerate(Zones):
        Poly = Polygon(zone)
        if Poly.contains(Point(BB[0], BB[1])):
            return i
    return 100

The following code is for the case only one time step is going to be passed through the model

In [ ]:
def scene_eval(Best_Model, Scene, Adj_Mat,Target, future, Zonecft):
    Pred = []
    total_time = 0
    NScene, GRUh0, inference_time = eval(Best_Model, Scene[:,-20:,:,:], Adj_Mat[:,:20,:32,:32], None, False)
    Pred.append(NScene)
    NScene = attach(NScene, Target[:,0,:,2:10], Zonecft).unsqueeze(0)
    # NScene = torch.cat((Scene[:,-19:,:,:],NScene), dim=1)
    
    for f in range(future-1):
        # NScene1, GRUh0, inference_time = eval(Best_Model, NScene, Adj_Mat[:,f+1:f+21,:32,:32], GRUh0[:,0,:].unsqueeze(1), False)
        NScene1, GRUh0, inference_time = eval(Best_Model, NScene, Adj_Mat[:,f+1,:32,:32].unsqueeze(1), GRUh0, False)
        Pred.append(NScene1)
        NScene1 = attach(NScene1, Target[:,f+1,:,2:10], Zonecft).unsqueeze(0)
        # NScene = torch.cat((NScene[:,-19:,:,:], NScene1), dim=1)
        total_time += inference_time

    Pred = torch.stack(Pred, dim=1).to(torch.int)
    tag = Target[0,:,:,:2].unsqueeze(0) != 0
    Pred = Pred*(tag.to(torch.int))

    loss = F.mse_loss(Pred, Target[0,:,:,:2].unsqueeze(0))
    msePtime = torch.sqrt(torch.sum((Pred - Target[0,:,:,:2].unsqueeze(0)) ** 2, dim = -1)).sum(dim = -1)/32
    msePuser = torch.sqrt(torch.sum((Pred - Target[0,:,:,:2].unsqueeze(0)) ** 2, dim = -1)).sum(dim = 1)/future

    # print(f"loss: {loss.item()} and inference time: {total_time} ms")
    # print(f"mse per time step: {msePtime}")
    # print(f"mse per user: {msePuser}")
    return Pred, loss, msePtime.squeeze(0), msePuser.squeeze(0), total_time

In [ ]:
def scene_eval20(Best_Model, Scene, Adj_Mat,Target, future, Zonecft):
    Pred = []
    total_time = 0
    NScene, GRUh0, inference_time = eval(Best_Model, Scene[:,-20:,:,:], Adj_Mat[:,:20,:32,:32], None, False)
    Pred.append(NScene)
    NScene = attach(NScene, Target[:,0,:,2:10], Zonecft).unsqueeze(0)
    NScene = torch.cat((Scene[:,-19:,:,:], NScene), dim=1)
    
    for f in range(future-1):
        NScene1, GRUh0, inference_time = eval(Best_Model, NScene, Adj_Mat[:,f+1:f+21,:32,:32], GRUh0[:,0,:].unsqueeze(1), False)
        Pred.append(NScene1)
        NScene1 = attach(NScene1, Target[:,f+1,:,2:10], Zonecft).unsqueeze(0)
        NScene = torch.cat((NScene[:,-19:,:,:], NScene1), dim=1)
        total_time += inference_time

    Pred = torch.stack(Pred, dim=1)
    tag = Target[0,:,:,:2].unsqueeze(0) != 0
    Pred = Pred*(tag.to(torch.int))

    loss = F.mse_loss(Pred, Target[0,:,:,:2].unsqueeze(0))
    msePtime = torch.sqrt(torch.sum((Pred - Target[0,:,:,:2].unsqueeze(0)) ** 2, dim = -1)).sum(dim = -1)/32
    msePuser = torch.sqrt(torch.sum((Pred - Target[0,:,:,:2].unsqueeze(0)) ** 2, dim = -1)).sum(dim = 1)/future

    # print(f"loss: {loss.item()} and inference time: {total_time} ms")
    # print(f"mse per time step: {msePtime}")
    # print(f"mse per user: {msePuser}")
    return Pred, loss, msePtime.squeeze(0), msePuser.squeeze(0), total_time

In [ ]:
future = 30
Prediction = []
totalmsePtime = torch.zeros(30, device=Adj_Mat_Scene.device)
totalmsePuser = torch.zeros(32, device=Adj_Mat_Scene.device)
total_loss = 0

total_time = 0
Adj_Mat = torch.cat((Adj_Mat_Scene, Adj_Mat_Target), dim=1).unsqueeze(0)
Zonecft = Zoneconf()
for n in range(250):
    Scene = ActualScene[n,1:,:32,:].unsqueeze(0)
    Target = ActualTarget[n,:,:32].unsqueeze(0)
    adj_mat = Adj_Mat[:,n,:,:32,:32]
    Pred, loss, msePtime, msePuser, elapsed_time = scene_eval(Best_Model, Scene, adj_mat, Target, future, Zonecft)
    totalmsePtime = totalmsePtime+ msePtime
    totalmsePuser = totalmsePuser+ msePuser
    total_loss += loss
    total_time += elapsed_time
    Prediction.append(Pred)

totalmsePtime = totalmsePtime/250
totalmsePuser = totalmsePuser/250
total_loss = total_loss/250
Prediction = torch.cat(Prediction, dim=0)
print(f"Total loss: {total_loss.item()}")
print(f"Total mse per time step: {totalmsePtime}")
print(f"Total mse per user: {totalmsePuser}")
print(f"Total inference time: {total_time/250} ms")
print("done")

In [ ]:
n_scene = 50
obj = 60
compare = torch.cat((Prediction[n_scene,:,obj], ActualTarget[n_scene,:,obj,:2]), dim=-1)
print(compare.to(torch.int32))

In [ ]:
preddata = Predicteddata.cpu()
acttarget = ActualTarget.cpu()
actscene = ActualScene.cpu()

# 41 different colors codes
color = torch.randint(0, 255, (41, 3), dtype=torch.uint8)
print(actscene.shape, acttarget.shape, preddata.shape)

In [ ]:
n= 2 # test sample number

i = 0
x = preddata[n][:,:32, 0]
y = preddata[n][:,:32, 1]
xs = actscene[n][:20,:32, 0]
ys = actscene[n][:20,:32, 1]
xr = acttarget[n][:20,:32, 10]
yr = acttarget[n][:20,:32, 11]
# Pliotting the predicted data
for keys in colors.keys():
    if i < 32:
        plt.scatter(xs[:,i], ys[:,i], color = colors[keys])
        plt.scatter(xr[:,i], yr[:,i], color = colors[keys])
        plt.scatter(x[:,i], y[:,i], color = colors[keys])
        # plt.scatter(xr[:,i], yr[:,i], color = 'red')
        # plt.scatter(x[:,i], y[:,i], color = 'black')
        plt.title('Predicted Data')
        i += 1




In [ ]:
# load an image and plot the ego and target on the image
import matplotlib.pyplot as plt
color = torch.randint(0, 255, (32, 3), dtype=torch.uint8)
img = cv2.imread('RawData/1.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
n = 30


xs = actscene[n][:,:32, 0]
ys = actscene[n][:,:32, 1]
xr = acttarget[n][:,:32, 0]
yr = acttarget[n][:,:32, 1]
for m in range(32):
    # add points to the coordinates on pred, grth and grth_target
    for i in range(20):
        cv2.circle(img, (int(xs[i,m]), int(ys[i,m])), 2, color[m].tolist(), -1)
        cv2.circle(img, (int(xr[i,m]), int(yr[i,m])), 4, color[m].tolist(), -1)
        # cv2.circle(img, (int(grth_target[i,0]), int(grth_target[i,1])), 4, (255,0,0), -1)
    for i in range(20, 30):
        cv2.circle(img, (int(xr[i,m]), int(yr[i,m])), 4, color[m].tolist(), -1)
#plot polygon on the image
# for i, zone in enumerate(ZoneConf):
#     cv2.polylines(img, [np.array(zone).astype(np.int32)], isClosed=True, color=(0,0,0), thickness=1)
plt.imshow(img)
plt.legend(['Predicted', 'Target'])

In [ ]:
colors = {
'aliceblue':            '#F0F8FF',
'antiquewhite':         '#FAEBD7',
'aqua':                 '#00FFFF',
'aquamarine':           '#7FFFD4',
'azure':                '#F0FFFF',
'beige':                '#F5F5DC',
'bisque':               '#FFE4C4',
'black':                '#000000',
'blanchedalmond':       '#FFEBCD',
'blue':                 '#0000FF',
'blueviolet':           '#8A2BE2',
'brown':                '#A52A2A',
'burlywood':            '#DEB887',
'cadetblue':            '#5F9EA0',
'chartreuse':           '#7FFF00',
'chocolate':            '#D2691E',
'coral':                '#FF7F50',
'cornflowerblue':       '#6495ED',
'cornsilk':             '#FFF8DC',
'crimson':              '#DC143C',
'cyan':                 '#00FFFF',
'darkblue':             '#00008B',
'darkcyan':             '#008B8B',
'darkgoldenrod':        '#B8860B',
'darkgray':             '#A9A9A9',
'darkgreen':            '#006400',
'darkkhaki':            '#BDB76B',
'darkmagenta':          '#8B008B',
'darkolivegreen':       '#556B2F',
'darkorange':           '#FF8C00',
'darkorchid':           '#9932CC',
'darkred':              '#8B0000',
'darksalmon':           '#E9967A',
'darkseagreen':         '#8FBC8F',
'darkslateblue':        '#483D8B',
'darkslategray':        '#2F4F4F',
'darkturquoise':        '#00CED1',
'darkviolet':           '#9400D3',
'deeppink':             '#FF1493',
'deepskyblue':          '#00BFFF',
'dimgray':              '#696969',
'dodgerblue':           '#1E90FF',
'firebrick':            '#B22222',
'floralwhite':          '#FFFAF0',
'forestgreen':          '#228B22',
'fuchsia':              '#FF00FF',
'gainsboro':            '#DCDCDC',
'ghostwhite':           '#F8F8FF',
'gold':                 '#FFD700',
'goldenrod':            '#DAA520',
'gray':                 '#808080',
'green':                '#008000',
'greenyellow':          '#ADFF2F',
'honeydew':             '#F0FFF0',
'hotpink':              '#FF69B4',
'indianred':            '#CD5C5C',
'indigo':               '#4B0082',
'ivory':                '#FFFFF0',
'khaki':                '#F0E68C',
'lavender':             '#E6E6FA',
'lavenderblush':        '#FFF0F5',
'lawngreen':            '#7CFC00',
'lemonchiffon':         '#FFFACD',
'lightblue':            '#ADD8E6',
'lightcoral':           '#F08080',
'lightcyan':            '#E0FFFF',
'lightgoldenrodyellow': '#FAFAD2',
'lightgreen':           '#90EE90',
'lightgray':            '#D3D3D3',
'lightpink':            '#FFB6C1',
'lightsalmon':          '#FFA07A',
'lightseagreen':        '#20B2AA',
'lightskyblue':         '#87CEFA',
'lightslategray':       '#778899',
'lightsteelblue':       '#B0C4DE',
'lightyellow':          '#FFFFE0',
'lime':                 '#00FF00',
'limegreen':            '#32CD32',
'linen':                '#FAF0E6',
'magenta':              '#FF00FF',
'maroon':               '#800000',
'mediumaquamarine':     '#66CDAA',
'mediumblue':           '#0000CD',
'mediumorchid':         '#BA55D3',
'mediumpurple':         '#9370DB',
'mediumseagreen':       '#3CB371',
'mediumslateblue':      '#7B68EE',
'mediumspringgreen':    '#00FA9A',
'mediumturquoise':      '#48D1CC',
'mediumvioletred':      '#C71585',
'midnightblue':         '#191970',
'mintcream':            '#F5FFFA',
'mistyrose':            '#FFE4E1',
'moccasin':             '#FFE4B5',
'navajowhite':          '#FFDEAD',
'navy':                 '#000080',
'oldlace':              '#FDF5E6',
'olive':                '#808000',
'olivedrab':            '#6B8E23',
'orange':               '#FFA500',
'orangered':            '#FF4500',
'orchid':               '#DA70D6',
'palegoldenrod':        '#EEE8AA',
'palegreen':            '#98FB98',
'paleturquoise':        '#AFEEEE',
'palevioletred':        '#DB7093',
'papayawhip':           '#FFEFD5',
'peachpuff':            '#FFDAB9',
'peru':                 '#CD853F',
'pink':                 '#FFC0CB',
'plum':                 '#DDA0DD',
'powderblue':           '#B0E0E6',
'purple':               '#800080',
'red':                  '#FF0000',
'rosybrown':            '#BC8F8F',
'royalblue':            '#4169E1',
'saddlebrown':          '#8B4513',
'salmon':               '#FA8072',
'sandybrown':           '#FAA460',
'seagreen':             '#2E8B57',
'seashell':             '#FFF5EE',
'sienna':               '#A0522D',
'silver':               '#C0C0C0',
'skyblue':              '#87CEEB',
'slateblue':            '#6A5ACD',
'slategray':            '#708090',
'snow':                 '#FFFAFA',
'springgreen':          '#00FF7F',
'steelblue':            '#4682B4',
'tan':                  '#D2B48C',
'teal':                 '#008080',
'thistle':              '#D8BFD8',
'tomato':               '#FF6347',
'turquoise':            '#40E0D0',
'violet':               '#EE82EE',
'wheat':                '#F5DEB3',
'white':                '#FFFFFF',
'whitesmoke':           '#F5F5F5',
'yellow':               '#FFFF00',
'yellowgreen':          '#9ACD32'}

In [ ]:
args.hidden_size

In [ ]:
import argparse
import os
import torch
import datetime
cwd = os.getcwd()
csvpath = os.path.join(cwd,'Processed','Trj20240506T2121.csv')
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
# # Hyperparameters
hidden_size, num_layersGAT, num_layersGRU, expansion,  n_heads = 128, 2,6, 2, 8

learning_rate, schd_stepzise, gamma, epochs, batch_size, patience_limit, clip= 2e-4, 20, 0.5, 300, 128, 40, 1

Headers = ['Frame', 'ID', 'BBx', 'BBy','W', 'L' , 'Cls','Tr1', 'Tr2', 'Tr3', 'Tr4', 'Zone', 'Xreal', 'Yreal']
Columns_to_keep = [2,3,4,5,6,7,8,9,10,11] #['BBx', 'BBy','W', 'L' , 'Cls','Tr1', 'Tr2', 'Tr3', 'Tr4', 'Zone']
#  ['Vx', 'Vy', 'heading',xc,yc, Rc,Rc, SinX,CosX,SinY,CosY, Sin2X,Cos2X, Sin2Y, Cos2Y, Sin3X, Cos3X, Sin3Y, Cos3Y, Sin4X, Cos4X, Sin4Y, Cos4Y]
Columns_to_Predict = [0,1] #['BBx', 'BBy','Xreal', 'Yreal','Vx', 'Vy']
xyid = [0, 1] # the index of x and y  of the Columns_to_Keep in the columns for speed calculation
TrfL_Columns = [7,8,9,10]
NFeatures = len(Columns_to_keep)
input_size = NFeatures + 6 + 8
output_size = len(Columns_to_Predict)
Nusers, NZones = 32, 9
Nnodes, NTrfL, sl, sw, sn  = Nusers+NZones, 4, 20, 1, 5
future = 30
Centre = [512,512]

sos = torch.tensor([1022,1022], device=device)
eos = torch.tensor([1021,1021], device=device)

generate_data = False #  Add class by loading the CSV file
# generate_data = True #  Add class by loading the CSV file
loadData = not generate_data      # load the class from the saved file
Train = True # It's for training the model with the prepared data
test_in_epoch = True # It's for testing the model in each epoch
model_from_scratch = True # It's for creating model from scratch
load_the_model = not model_from_scratch # It's for loading model
Seed = loadData # If true, it will use the predefined seed to load the indices
only_test = False # It's for testing the model only
concat = False



LR = [learning_rate] #, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0]
# LR = [1e-2]
HS = [128] #, 32, 64, 128, 256, 512, 1024]
NL = [num_layersGAT] #, 2, 3, 4]
BS = [batch_size] #, 32, 64, 128, 256, 512, 1024]

csvpath ='/home/abdikhab/New_Idea_Traj_Pred/RawData/TrfZonXYCam.csv'
Zoneconf_path ='/home/abdikhab/New_Idea_Traj_Pred/utilz/ZoneConf.yaml'
dataset_path = os.path.join(os.getcwd(), 'Pickled')
ct = datetime.datetime.now().strftime("%m-%d-%H-%M")
print(f"Total  # of GPUs {torch.cuda.device_count()}")
# spread the model to multiple GPUs
print(f"Using {device} device")


# Use argparse to get the parameters
parser = argparse.ArgumentParser(description='Trajectory Prediction')
parser.add_argument('--ct', type=str, default=ct, help='Current time')
parser.add_argument('--Zonepath', type=str, default=Zoneconf_path, help='Zone Conf path')
parser.add_argument('--Nfeatures', type=int, default=NFeatures, help='Number of features')
parser.add_argument('--Nnodes', type=int, default=Nnodes, help='Number of nodes')
parser.add_argument('--NZones', type=int, default=NZones, help='Number of zones')
parser.add_argument('--NTrfL', type=int, default=NTrfL, help='Number of traffic lights')
parser.add_argument('--sl', type=int, default=sl, help='Sequence length')
parser.add_argument('--future', type=int, default=future, help='Future length')
parser.add_argument('--sw', type=int, default=sw, help='Sliding window')
parser.add_argument('--sn', type=int, default=sn, help='Sliding number')
parser.add_argument('--Columns_to_keep', type=list, default=Columns_to_keep, help='Columns to keep')
parser.add_argument('--Columns_to_Predict', type=list, default=Columns_to_Predict, help='Columns to predict')
parser.add_argument('--TrfL_Columns', type=list, default=TrfL_Columns, help='Traffic light columns')
parser.add_argument('--Nusers', type=int, default=Nusers, help='Number of maneuvers')
parser.add_argument('--sos', type=int, default=sos, help='Start of sequence')
parser.add_argument('--eos', type=int, default=eos, help='End of sequence')
parser.add_argument('--xyidx', type=list, default=xyid, help='X and Y index')
parser.add_argument('--Centre', type=list, default=Centre, help='Centre')

parser.add_argument('--input_size', type=int, default=input_size, help='Input size')
parser.add_argument('--hidden_size', type=int, default=hidden_size, help='Hidden size')
parser.add_argument('--num_layersGAT', type=int, default=num_layersGAT, help='Number of layers')
parser.add_argument('--num_layersGRU', type=int, default=num_layersGRU, help='Number of layers')
parser.add_argument('--output_size', type=int, default=output_size, help='Output size')
parser.add_argument('--n_heads', type=int, default=n_heads, help='Number of heads')
parser.add_argument('--concat', type=bool, default=concat, help='Concat')
parser.add_argument('--dropout', type=float, default=0.001, help='Dropout')
parser.add_argument('--leaky_relu_slope', type=float, default=0.2, help='Leaky relu slope')
parser.add_argument('--expansion', type=int, default=expansion, help='Expantion')


parser.add_argument('--epochs', type=int, default=epochs, help='Number of epochs')
parser.add_argument('--learning_rate', type=float, default=learning_rate, help='Learning rate')
parser.add_argument('--batch_size', type=int, default=batch_size, help='Batch size')
parser.add_argument('--patience_limit', type=int, default=patience_limit, help='Patience limit')
parser.add_argument('--schd_stepzise', type=int, default=schd_stepzise, help='Scheduler step size')
parser.add_argument('--gamma', type=float, default=gamma, help='Scheduler Gamma')


parser.add_argument('--only_test', type=bool, default=only_test, help='Only test')
parser.add_argument('--generate_data', type=bool, default=generate_data, help='Generate data')
parser.add_argument('--loadData', type=bool, default=loadData, help='Load data')
parser.add_argument('--Train', type=bool, default=Train, help='Train')
parser.add_argument('--test_in_epoch', type=bool, default=test_in_epoch, help='Test in epoch')
parser.add_argument('--model_from_scratch', type=bool, default=model_from_scratch, help='Model from scratch')
parser.add_argument('--load_the_model', type=bool, default=load_the_model, help='Load the model')
parser.add_argument('--Seed', type=bool, default=Seed, help='Seed')
parser.add_argument('--device', type=str, default=device, help='device')

args, unknown = parser.parse_known_args()

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

nx_graph = nx.from_edgelist(edgindx)
fig = plt.figure(figsize=(10, 10))
nx.draw(nx_graph, with_labels=True)

In [ ]:
color[0].tolist()

In [ ]:
#Check if the target is in the zone
for i in range(len(grth)):
    print(zonefinder(grth[i,0]*1024,grth[i,1]*1024, ZoneConf))

In [ ]:
# Plotting all of the predicted data
for i in range(20, 21):
    plt.plot(Predicteddata[-1][i,:,0].to('cpu'), Predicteddata[-1][i,:,1].to('cpu'), 'r')
    plt.plot(ego_gt[i][:,0], ego_gt[i][:,1], 'b')

# plt.scatter(x,y, marker='x', color='r')
# plt.scatter(gyx, gyy, marker='*', color='b')
#plt.scatter(groundtruthx[i][:,0], groundtruthx[i][:,1], marker='o', color='g')

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
import os
import math
from convlstm import ConvLSTMCell
from utils import *
from lstm_traj import *
from LSTM import *
#from socialconv.model import highwayNet
#from torchviz import make_dot
# Load the data
cwd = os.getcwd()
csvpath = os.path.join(cwd,'Processed','Trj20240111T2201.csv')
# # Hyperparameters
input_size, hidden_size, num_layers, output_size, epochs, learning_rate = 10, 64, 4, 1, 200, 1e-2
sequence_length, sw , shift, batch_size = 100, 10, 8, 1
sl = sequence_length
add_class = True #  Add class by loading the CSV file
load_class = not add_class      # load the class from the saved file
Train = True # It's for training the model with the prepared data
test_in_epoch = True # It's for testing the model in each epoch
model_from_scratch = True # It's for creating model from scratch
load_the_model = not model_from_scratch # It's for loading model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Create datasets

if add_class:
    df = loadcsv(csvpath)
    dataset, maxdata, mindata = def_class(df, sw, sl, shift, output_size, input_size, normalize = False)
    
if load_class:
    dataset = torch.load(os.path.join(os.getcwd(),'Pickled','datasetTorch.pt'))
    normpar = torch.load(os.path.join(os.getcwd(),'Pickled','normpar.pt'))
    print('dataset loaded')
print("Dataset total length is", len(dataset))

# Create train and test sets
train_loader, test_loader, val_loader = prep_data(dataset, batch_size)

**This part Summarizes the txt files**

In [ ]:
# load the log.txt file and append to txt
txt = []
tmp = "Hellp"
n=0
j = 0
with open(os.path.join(os.getcwd(),'logs', 'log-05-07-19-13.txt'), 'r') as file:
    logs = file.read()
    for log in logs.split('\n'):
        if tmp.startswith('Average') and log.startswith(' Accuracy'):
            tmp1 = log
            tmp2 = tmp
        if log.startswith('Learning'):
            j += 1
            if j == 1:
                txt.append(log)
            else:
                txt.append(tmp1)
                txt.append(tmp2) 
                txt.append(log)
        tmp = log

with open(os.path.join(os.getcwd(),'logs', 'Summary of log-05-07-19-13.txt'), 'w') as file:
    for log in txt:
        file.write(log + '\n')